In [1]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn import tree
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import gc
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import StandardScaler

train_path = '/home/zhangyong/Documents/data/kaggle/santander_value/train.csv'
test_path = '/home/zhangyong/Documents/data/kaggle/santander_value/test.csv'


dat = pd.read_csv(train_path)
test = pd.read_csv(test_path)
y = dat['target']
X = dat.drop(['ID', 'target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [2]:
standardized_train = StandardScaler().fit_transform(X.values)

In [3]:
train = X
target = y
feature_df = X.describe().T
feature_df = feature_df.reset_index().rename(columns = {'index' : 'columns'})
feature_df['distinct_vals'] = feature_df['columns'].apply(lambda x : len(train[x].value_counts()))
feature_df['column_var'] = feature_df['columns'].apply(lambda x : np.var(train[x]))
feature_df['column_std'] = feature_df['columns'].apply(lambda x : np.std(train[x]))
feature_df['column_mean'] = feature_df['columns'].apply(lambda x : np.mean(train[x]))
feature_df['target_corr'] = feature_df['columns'].apply(lambda x : np.corrcoef(target, train[x])[0][1])

In [5]:
feature_df.head()

,columns,count,mean,std,min,25%,50%,75%,max,distinct_vals,column_var,column_std,column_mean,target_corr
0,48df886f9,4459.0,14654.930101,3.893298e+05,0.0,0.0,0.0,0.0,20000000.0,32,1.515437e+11,3.892862e+05,14654.930101,0.010188
1,0deb4b6a8,4459.0,1390.894819,6.428302e+04,0.0,0.0,0.0,0.0,4000000.0,5,4.131381e+09,6.427582e+04,1390.894819,0.013805
2,34b15f335,4459.0,26722.450922,5.699652e+05,0.0,0.0,0.0,0.0,20000000.0,29,3.247875e+11,5.699013e+05,26722.450922,0.014694
3,a8cb14b00,4459.0,4530.163714,2.359124e+05,0.0,0.0,0.0,0.0,14800000.0,3,5.564218e+10,2.358860e+05,4530.163714,-0.002917
4,2f0771a37,4459.0,26409.957390,1.514730e+06,0.0,0.0,0.0,0.0,100000000.0,6,2.293893e+12,1.514560e+06,26409.957390,0.016647


In [7]:
feature_df = feature_df.sort_values('column_var', ascending = True)
feature_df.head()

,columns,count,mean,std,min,25%,50%,75%,max,distinct_vals,column_var,column_std,column_mean,target_corr
1294,0db8a9272,4459.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,NaN
1212,87eb21c50,4459.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,NaN
1231,5ea313a8c,4459.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,NaN
1252,0987a65a1,4459.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,NaN
3840,2e52b0c6a,4459.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,NaN


In [9]:
feature_df['column_var'] = (feature_df['column_var'] - feature_df['column_var'].min()) / (feature_df['column_var'].max() - feature_df['column_var'].min())

In [17]:
%matplotlib inline
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

trace1 = go.Scatter(x=feature_df['columns'], y=feature_df['column_var'], opacity=0.75, marker=dict(color="red"))
layout = dict(height=400, title='Feature Variance', legend=dict(orientation="h"));
fig = go.Figure(data=[trace1], layout=layout);
iplot(fig);


In [12]:
import matplotlib